In [ ]:
# default_exp core

# Pyopticum

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import os

### First of all lets do some imports

In [ ]:
#export
#nbdev_comment from __future__ import print_function
from typing import overload
import random

import pint

#from functools import singledispatch 
from multipledispatch import dispatch

import math
import inspect
import os
import pytest
import ipytest
ipytest.autoconfig()


try:
      from IPython.display import display, Math, Latex, HTML, Markdown
      display(HTML("Using Ipython"))
      ascii_print = print
      def print(*args, **kwargs):
        for item in args:
            display(item)

except ImportError:
      from bs4 import BeautifulSoup
      from markdown import markdown
      print("unable to import IPython, ignoring, using basic print")


### Lets set up Pint with some useful extra entities

#export
ureg = pint.UnitRegistry()
ureg.define('Circle_of_confusion = [] = coc')
ureg.define('Aperture = [] = fnumber')

### Define a base class that make POD members availible at top layer as attributes

In [ ]:
#export
class Help_system:
    def __init__(self):
        pass

    def initialize_help_system(self):
        #print(f"createing help system")
        self.__dict__['ureg'] = pint.UnitRegistry()
        self.__dict__['is_ipython'] = self.__is_running_under_ipython()
        if self.is_ipython:
            self.ureg.default_format = "L"
        else:
            self.ureg.default_format = "P"
        self.__dict__["echo_params"] = False

    def __is_running_under_ipython(self):
        try:
            get_ipython
            return True
        except:
            return False

    def wrap_unit(self, eqn, style):
        if (self.is_ipython):
            if style.lower() == "math":
                return Math(eqn)
            if style.lower() == "html":
                return HTML(eqn)
            if style.lower() == "latex":
                return Latex(eqn)
            if style.lower() == "markdown":
                return Markdown(eqn)
        else:
            if style.lower() == "html":
                soup = BeautifulSoup(eqn)
                return soup.get_text()
            if style.lower() == "markdown":
                html = markdown(eqn)
                soup = BeautifulSoup(html, features='html.parser')
                return soup.get_text()
        return eqnprint

    def help(self, command):
        """
        runs the help function for the command
        """
        method_to_call = getattr(self, command+'_help')
        result = method_to_call()
    
    def noisy_params(self):
        self.__dict__['echo_params'] = True
        
    def silence_params(self):
        self.__dict__['echo_params'] = False

In [ ]:
#export
class Lens(Help_system):
    class Raw_lens_data():
        def __init__(self):
            self.focal_length = 0*ureg.mm
            self.aperture = 0*ureg.Aperture
            self.focus_distance = 0 * ureg.meters
            


    @ureg.wraps(None,(None,'mm','Aperture','m'))
    def __init__(self, focal_length=40*ureg.mm, aperture=2.0*ureg.Aperture, focus_distance=4.0*ureg.meters):
        # system setup
        self.initialize_help_system()
        self.__dict__['class_preamble'] = self.__class__.__name__
        self.__dict__['pod_name'] = "lens_data"
        self.__dict__[self.pod_name] = self.Raw_lens_data()

        self.lens_data.focal_length = focal_length*ureg.mm
        self.lens_data.aperture = aperture*ureg.Aperture
        self.lens_data.focus_distance = focus_distance*ureg.meters

    @property
    @ureg.wraps('mm',(None))
    def focal_length(self):
        return self.lens_data.focal_length
    
    @focal_length.setter
    def focal_length(self,args):
        if isinstance(args,(int,float)):
            self.lens_data.focal_length = args*ureg.mm
            return
        try:
            if not (args.check('[length]')):
                raise Exception(f"die setter is not suitable quantity (mm) {args}")
            self.lens_data.focal_length = args
        except:
            raise Exception(f"die setter is not suitable quantity (mm) {args}")
            
    @property
    @ureg.wraps('Aperture',(None))
    def aperture(self):
        return self.lens_data.aperture
    
    @aperture.setter
    def aperture(self,inargs):
        if isinstance(inargs,(int,float)):
            self.lens_data.aperture = inargs*ureg.Aperture
            return
        if isinstance(inargs, ureg.Aperture):
            self.lens_data.aperture = inargs
        else:
            raise Exception(f"aperture is not suitable quantity (Aperture) {inargs}")
    
    @property
    @ureg.wraps('mm',(None))
    def focus_distance(self):
        return self.lens_data.focus_distance
    
    @focus_distance.setter
    def focus_distance(self, inargs):
        if isinstance(inargs,(int,float)):
            self.lens_data.focus_distance = inargs*ureg.m
            return
        try:
            if not (args.check('[length]')):
                raise Exception(f"focus_distance is not suitable quantity (m) {inargs}")
            self.lens_data.focus_distance = inargs
        except:
            raise Exception(f"focus_distance is not suitable quantity (m) {inargs}")



In [ ]:
#export
class Sensor(Help_system):
    """
    class to hold sensor data and methods
    """
    class Raw_sensor_data:
        def __init__(self):
            self.die_size = (0,0)
            self.pixel_size = (0,0)
            self.diagonal = 0
            self.circle_of_confusion = 0
            self.circle_of_confusion_method = "Modern"

    @ureg.wraps(None, (None, 'mm','mm','micrometer','micrometer'))
    def __init__(self, die_size_x,die_size_y, pixel_size_x,pixel_size_y):
        # system setup
        self.initialize_help_system()
        self.__dict__['class_preamble'] = self.__class__.__name__
        self.__dict__['pod_name'] = "sensor_data"
        self.__dict__[self.pod_name] = self.Raw_sensor_data()
        #class variable setup
        self.sensor_data.die_size = (die_size_x*ureg.mm,   die_size_y*ureg.mm)
        self.sensor_data.diagonal = self.diagonal
        self.sensor_data.pixel_size = (pixel_size_x *ureg.micrometer,pixel_size_y * ureg.micrometer)
        self.sensor_data.circle_of_confusion_method = "Modern"
        self.calculate_circle_of_confusion()

    @property
    def die_size(self):
        """
        the size of the physical sensor
        """
        return self.sensor_data.die_size
    
    @die_size.setter
    def die_size(self,args):
        if len(args) > 2:
            raise Exception("die_size args should size 2, (x,y) ")
        in_x = args[0]
        in_y = args[1]
        try:
            if not (in_x.check('[length]') & in_y.check('[length]')):
                raise Exception(f"die setter is not suitable quantity (mm,mm) {in_x},{in_y}")
        except:
            raise Exception(f"die setter is not suitable quantity (mm,mm) {in_x},{in_y}")

        self.sensor_data.die_size = (in_x,in_y)
        
    
    @die_size.deleter
    def die_size(self):
        self.sensor_data.die_size = (None, None)
        self.sensor_data.diagonal = None
    
    @property
    @ureg.wraps('mm',(None))
    def diagonal(self):
        """
        The diagonal size of the sensor (re-calculated upon calling)
        """
        self.sensor_data.diagonal = math.sqrt( math.pow(self.sensor_data.die_size[0].magnitude,2) 
                                              + math.pow(self.sensor_data.die_size[1].magnitude,2) )*ureg.mm
        return self.sensor_data.diagonal
    
    @diagonal.setter
    def diagonal(self,args):
        raise Exception("Diagonal is Read Only")
        
    @diagonal.deleter
    def diagonal(self):
        self.sensor_data.diagonal = None
    
    @property
    def pixel_size(self):
        return self.sensor_data.pixel_size

    @pixel_size.setter
    def pixel_size(self, args):
        if len(args) > 2:
            raise Exception("pixel_size args should size 2, (x,y) ")
        in_x = args[0]
        in_y = args[1]
        try:
            if not (in_x.check('[length]') & in_y.check('[length]')):
                raise Exception(f"die setter is not suitable quantity (um,um) {in_x},{in_y}")
        except:
            raise Exception(f"die setter is not suitable quantity (um,um) {in_x},{in_y}")

        self.sensor_data.pixel_size = (in_x,in_y)
        
    
    @pixel_size.deleter
    def pixel_size(self):
        self.sensor_data.pixel_size = (None,None)
        self.sensor_data.circle_of_confusion = None 

                
    @property
    def circle_of_confusion_method(self):
        return self.sensor_data.circle_of_confusion_method
    
    @circle_of_confusion_method.setter
    def circle_of_confusion_method(self, in_args ):
        #method="modern",*args, **kwargs):
        if isinstance(in_args,str):
            method = in_args
        elif len(in_args) == 2:
            method = in_args[0]
            focus_object = in_args[1]
        else:
            raise LookupError(f"Too many arguments in the supplied arguments [{len(in_args)}] {in_args}")
        
        coc_method = [ "modern", "zeiss","kodak", "archaic" ]
        if method not in coc_method:
            raise ValueError(f"Circle of confusion method not supported [{method}]")
        self.sensor_data.circle_of_confusion_method = method
        
        
        if method == "kodak":
            self.calculate_circle_of_confusion(focus_object)
        else:
            self.calculate_circle_of_confusion()
        

    @circle_of_confusion_method.deleter
    def circle_of_confusion_method(self):
        self.sensor_data.circle_of_confusion_method = "modern"
        self.sensor_data.circle_of_confusion = None
                
    
    def circle_of_confusion_help(self)->None:
        print(HTML("<H2>Circle of Confusion</H2>"))
        print(HTML("Modern, Standard Method (Default)	Frame’s diagonal / 1500"))
        print(HTML("Zeiss, Formula	Frame’s diagonal / 1730"))
        print(HTML("Kodak, Formula	Focal length / 1720"))
        print(HTML("Archaic, Standard	Frame’s diagonal / 1000"))
        print(HTML("<code> circle_confusion(frame_diagonal,focal_length,method = 'modern')</code>"))
        return 1

    @property
    def circle_of_confusion(self):
        if (self.sensor_data.circle_of_confusion is None):
            raise Exception(f" Circle of confusion has not been calculated yet")
        
        return self.sensor_data.circle_of_confusion 
    
    
    @dispatch( ureg.Quantity )
    def calculate_circle_of_confusion(self, inarg):
        #ok we have been given a lens length
        if self.sensor_data.circle_of_confusion_method.lower() != "kodak":
            raise Exception("Given a lens length, but circle of confusion method is not kodak")
        self.__calculate_circle_of_confusion(inarg)
    
    @dispatch( Lens )
    def calculate_circle_of_confusion(self, inarg):
        self.__calculate_circle_of_confusion(inarg)
        
    @dispatch()
    def calculate_circle_of_confusion(self):
        self.__calculate_circle_of_confusion(0*ureg.mm)
        
    def __calculate_circle_of_confusion(self, inarg):
        
        def modern(frame_diagonal, ignore):
            return frame_diagonal/1500
        def zeiss(frame_diagonal, ignore):
            return frame_diagonal/1730
        def kodak(ignore, focal_length):
            return focal_length/1720
        def archaic(frame_diagonal, ignore):
            return frame_diagonal/1000
        
        coc_method = { "modern": modern,
                      "zeiss": zeiss,
                      "kodak": kodak,
                      "archaic": archaic}
        focal_length = 0
        if isinstance(inarg,Lens):
            focal_length = inarg.focal_length
        elif(self.sensor_data.circle_of_confusion_method.lower() == 'kodak' ):
            try:
                if not (inarg.check('[length]')):
                    raise Exception(f"focal length is not suitable quantity (mm) {inarg}")
            except:
                raise Exception(f"focal length is not suitable quantity (mm) {inarg}")
            #ok looks like we can use this as a focal length
            focal_length = in_arg
            
        if self.sensor_data.circle_of_confusion_method.lower() not in coc_method:
            raise ValueError(f"Unknown Circle of Confusion Method {method}")

        #if focal_length
        coc = coc_method.get(self.sensor_data.circle_of_confusion_method.lower())(self.sensor_data.diagonal,focal_length)
        self.sensor_data.circle_of_confusion = (coc.magnitude) * ureg['mm']
    
    @circle_of_confusion.deleter
    def circle_of_confusion(self):
        self.sensor_data.circle_of_confusion = None
        
    
                

In [ ]:
#export 
class Camera(Help_system):
    class Camera_data:
        def __init__(self):
            self.angle_of_view = (None,None)
            self.field_of_view = (None,None)
            
    def __init__(self, sensor: Sensor, lens: Lens):
        # system setup
        self.initialize_help_system()
        self.__dict__['class_preamble'] = self.__class__.__name__
        self.__dict__['pod_name'] = "camera_data"
        self.__dict__[self.pod_name] = self.Camera_data()
        
        self.sensor = sensor
        self.lens = lens
    
    def replace_lens(self, inlens: Lens):
        if not isinstance(inlens, Lens):
            raise Exception(f" passed object is not a lens")
        self.lens = inlens
    
    def replace_senspr(self, insensor: Sensor):
        if not isinstance(insensor, Sensor):
            raise Exception(f" passed object is not a sensor")
        self.sensor = insensor
        
    def angle_of_view_help(self):
        print(HTML("<h2>Angle of view</h2>"))
        print(HTML("<span>Figures out the witdh of the angle of view, given sensor / lens characteristics.</span>"))
        print(self.wrap_unit(r"\theta=2\cdot\arctan\left(\frac{h(s-f)}{2sf}\right)","math"))
        print(HTML("<code> angle_of_view(frame_dimension, focal_length, focus_distance)</code>"))

    @ureg.wraps(('radians','radians'),(None))
    def angle_of_view(self):
        aov_x = 2.0 * math.atan(((self.sensor.die_size[0] * (self.lens.focus_distance - self.lens.focal_length))/(2*self.lens.focus_distance*self.lens.focal_length)).magnitude)
        aov_y = 2.0 * math.atan(((self.sensor.die_size[1] * (self.lens.focus_distance - self.lens.focal_length))/(2*self.lens.focus_distance*self.lens.focal_length)).magnitude)
        self.camera_data.angle_of_view = (aov_x * ureg.radians ,aov_y * ureg.radians)
        return self.camera_data.angle_of_view
  
    def field_of_view_help(self):
        print(self.wrap_unit("<h2>Field of View</h2>","html"))
        print(self.wrap_unit("<a> implementation of this equation </a>","html"))
        print(self.wrap_unit(r'w=2s\cdot\tan\left(\frac{\theta}{2}\right)',"Math"))

    @ureg.wraps(('m','m'),(None))
    def field_of_view(self):
        if self.camera_data.angle_of_view[0] is None:
            self.angle_of_view()
        view_width_x = 2*self.lens.focus_distance*math.tan( self.camera_data.angle_of_view[0] / 2.0)
        view_width_y = 2*self.lens.focus_distance*math.tan( self.camera_data.angle_of_view[1] / 2.0)
        print(f" view width {view_width_x},{view_width_y}")
        self.camera_data.field_of_view = (view_width_x, view_width_y)
        return self.camera_data.field_of_view

    def hyperfocal_distance_help(self):
        print(HTML("<H2>Hyperfocal Distance</H2>"))
        print(HTML("<span> calculates hyperfocal distance"))
        print(self.wrap_unit(r"H=\frac{f^{2}}{N\cdot{c}}+f","Math"))

    @ureg.wraps(ureg.meters, (None))
    def hyperfocal_distance(self):
        # aperture for us is unitless but really is should be in 1/m 
        # TODO fix aperture to be correct units
        bottom = (1/(self.lens.aperture.magnitude*self.sensor.circle_of_confusion.magnitude))
        hfd = (pow(self.lens.focal_length.magnitude,2)/(bottom)+self.lens.focal_length.magnitude)
        return hfd*ureg('m')
    
    def near_DOF_limit_help(self):
        print(HTML("<H2>Near Depth of Field</H2>"))
        print(self.wrap_unit(r"D_{n}=\frac{s(H-f)}{H+s-2f}","Math"))
        
    @ureg.wraps(ureg.meters,(None))
    def near_DOF_limit(self):
        top = self.lens.focus_distance*(self.hyperfocal_distance() -self.lens.focal_length)
        bottom = self.hyperfocal_distance() + self.lens.focus_distance - 2*self.lens.focal_length
        return top/bottom
    
    def far_DOF_limit_help(self):
        print(HTML("<H2>Far Depth of Field</H2>"))
        print(self.wrap_unit(r"D_{f}=\frac{s(H-f)}{H-s}","Math"))
    
    @ureg.wraps(ureg.meters,(None))
    def far_DOF_limit(self):
        top = self.lens.focus_distance*(self.hyperfocal_distance() -self.lens.focal_length)
        bottom = self.hyperfocal_distance() - self.lens.focus_distance
        return top/bottom
    
    def DOF_help(self):
        print(HTML("<H2>Depth of Field</H2>"))
        print(self.wrap_unit(r"DOF = Far Limit - Near Limt","Math"))
        
    @ureg.wraps(ureg.meters,(None))
    def DOF(self):
        return self.far_DOF_limit() - self.near_DOF_limit()
    

In [ ]:
#export
class aPyopticum(Help_system):
    """
    Initial class to hold optical formula's etc

    """

    ###################################################################################################################
    def __init__(self, echo_params = False):
        #system setup
        self.initialize_help_system()
        #direct setup
        self.data = []
        self.echo_params = echo_params



    def about(self):
        """
        about this library and usage
        """
        print(self.wrap_unit("<h1>Pyopticum</h1>","html"))
        print(self.wrap_unit("see <a href='https://github.com/jlovick/Pyopticum'> Pyopticum </a> for source","html"))

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.


In [ ]:
pc = aPyopticum()
pc.about()

In [ ]:
def test_sensor_is_behaving():
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    da = mysensor.diagonal 
    mysensor.die_size  = ( 12*ureg.mm,  44*ureg.mm)
    db = mysensor.diagonal
    print(f"{mysensor.die_size} {da} --> {db}")
    

In [ ]:
test_sensor_is_behaving()

"(<Quantity(12, 'millimeter')>, <Quantity(44, 'millimeter')>) 17.029386365926403 millimeter --> 45.60701700396552 millimeter"

In [ ]:
def test_can_create_lens():
    mylens = Lens((40 * ureg.mm) ,(2.4*ureg.Aperture))

In [ ]:

def test_cant_create_lens_without_unit():
    with pytest.raises(Exception) as e_info:
        mylens = Lens(40,2.4)

In [ ]:
def test_focal_length_has_mm_units():
    mylens = Lens((40 * ureg.mm) ,(2.4*ureg.Aperture))
    assert mylens.focal_length == (40*ureg.mm)


In [ ]:
def test_can_create_sensor():
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)

In [ ]:
def test_sensor_diagonal():
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    print(f"die size = {mysensor.die_size}\n cell size = {mysensor.pixel_size}  ")
    
    assert mysensor.diagonal == 17.029386365926403 * ureg.mm
test_sensor_diagonal()

"die size = (<Quantity(13.0, 'millimeter')>, <Quantity(11.0, 'millimeter')>)\n cell size = (<Quantity(2.7, 'micrometer')>, <Quantity(2.7, 'micrometer')>)  "

In [ ]:
def test_circle_of_confusion_default():
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    print(mysensor.circle_of_confusion)
    assert mysensor.circle_of_confusion == (0.011352924243950934 * ureg.mm)
    
test_circle_of_confusion_default()

0.011352924243950934 <Unit('millimeter')>

In [ ]:
def test_archaic_circle_of_confusion():
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mylens = Lens((40 * ureg.mm) ,(2.0*ureg.Aperture))
    mysensor.circle_of_confusion_method = ("archaic", mylens)
    print(f" archaic coc = {mysensor.circle_of_confusion}")
    assert mysensor.circle_of_confusion == 0.017029386365926404*ureg.mm

test_archaic_circle_of_confusion()

' archaic coc = 0.017029386365926404 millimeter'

In [ ]:
def test_zeiss_circle_of_confusion():
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mysensor.circle_of_confusion_method = "zeiss"
    print(f" zeiss coc = {mysensor.circle_of_confusion}")
    assert mysensor.circle_of_confusion == 0.009843575934061504*ureg.mm
test_zeiss_circle_of_confusion()

' zeiss coc = 0.009843575934061504 millimeter'

In [ ]:
def test_can_create_camera():
    mylens = Lens(40*ureg.mm, 2.0*ureg.Aperture)
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mycamera = Camera(sensor = mysensor, lens = mylens)

In [ ]:
def test_camera_angle_of_view_help():
    mylens = Lens(40*ureg.mm, 2.0*ureg.Aperture)
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mycamera = Camera(sensor = mysensor, lens = mylens)
    mycamera.angle_of_view_help()

In [ ]:
def test_camera_angle_of_view():
    mylens = Lens(40*ureg.mm, 2.0*ureg.Aperture, 7.0*ureg.m)
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mycamera = Camera(sensor = mysensor, lens = mylens)
    (horizontal_aov,vertical_aov) =  mycamera.angle_of_view()
    assert math.isclose(horizontal_aov, 0.32037417920340566*ureg.radians, rel_tol=1e-6, abs_tol = 0.0)
    assert math.isclose(vertical_aov, 0.27174389171340224*ureg.radians, rel_tol=1e6,abs_tol = 0.0)
    print(f"{horizontal_aov},{vertical_aov}")


In [ ]:
def test_camera_field_of_view():
    mylens = Lens(40*ureg.mm, 2.0*ureg.Aperture, 7.0*ureg.m)
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mycamera = Camera(sensor = mysensor, lens = mylens)
    fov = mycamera.field_of_view()
    print(f"{fov}")
    assert math.isclose(fov[0].magnitude, 2.262, rel_tol=1e-6, abs_tol=0.0)
    assert math.isclose(fov[1].magnitude, 1.9139999999999998, rel_tol=1e-6, abs_tol=0.0)


In [ ]:
def test_camera_hyperfocal_distance():
    mylens = Lens(40*ureg.mm, 2.0*ureg.Aperture)
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mycamera = Camera(sensor = mysensor, lens = mylens)
    print(f" {mysensor.circle_of_confusion}")
    hyper_distance =mycamera.hyperfocal_distance()
    print(f"{hyper_distance}")
    assert math.isclose(hyper_distance.magnitude,(76.32935),rel_tol=1e-5,abs_tol=0.0)

In [ ]:
def test_camera_near_focal_distance():
    mylens = Lens(40*ureg.mm, 2.0*ureg.Aperture, 4.0 * ureg['meters'])
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mycamera = Camera(sensor = mysensor, lens = mylens)
    print(f" {mysensor.circle_of_confusion}")
    nfd  = mycamera.near_DOF_limit()
    print(f"{nfd}")
    assert math.isclose(nfd.magnitude,3.8026, rel_tol=1e-5, abs_tol=0.0)


In [ ]:
def test_camera_far_focal_distance():
    mylens = Lens(40*ureg.mm, 2.0*ureg.Aperture, 4.0 * ureg['meters'])
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mycamera = Camera(sensor = mysensor, lens = mylens)
    print(f" {mysensor.circle_of_confusion}")
    ffd  = mycamera.far_DOF_limit()
    print(f"{ffd}")
    assert math.isclose(ffd.magnitude,4.218998, rel_tol =1e-5, abs_tol=0.0)


In [ ]:
def test_camera_DOF():
    mylens = Lens(40*ureg.mm, 2.0*ureg.Aperture, 4.0 * ureg['meters'])
    mysensor = Sensor(13*ureg.mm,11*ureg.mm, 2.7*ureg.micrometer, 2.7*ureg.micrometer)
    mycamera = Camera(sensor = mysensor, lens = mylens)
    print(f" {mysensor.circle_of_confusion}")
    dof  = mycamera.DOF()
    print(f"{dof}")
    assert math.isclose(dof.magnitude,0.416382,rel_tol=1e-5, abs_tol=0.0)


In [ ]:
ipytest.run()

"(<Quantity(12, 'millimeter')>, <Quantity(44, 'millimeter')>) 17.029386365926403 millimeter --> 45.60701700396552 millimeter"

.....

"die size = (<Quantity(13.0, 'millimeter')>, <Quantity(11.0, 'millimeter')>)\n cell size = (<Quantity(2.7, 'micrometer')>, <Quantity(2.7, 'micrometer')>)  "

.

0.011352924243950934 <Unit('millimeter')>

...

'4.218998212203659 meter'

.

' 0.011352924243950934 millimeter'

'0.41638296988327417 meter'

.                                                                            [100%]
17 passed in 6.81s
